<a href="https://colab.research.google.com/github/SonaArajyan03/Personal_Finance_Manager/blob/main/personalfinancemanager2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import csv

class Transaction:
    def __init__(self, amount, date, description):
        self.amount = amount
        self.date = date
        self.description = description

    def get_attributes(self):
        return [self.amount, self.date, self.description]


class Expense(Transaction):
    def __init__(self, amount, date, description):
        super().__init__(amount, date, description)


class OneTimePurchase(Expense):
    def __init__(self, amount, date, description):
        super().__init__(amount, date, description)


class Subscription(Expense):
    def __init__(self, amount, date, description):
        super().__init__(amount, date, description)


class Income(Transaction):
    def __init__(self, amount, date, description):
        super().__init__(amount, date, description)


class OneTimePayment(Income):
    def __init__(self, amount, date, description):
        super().__init__(amount, date, description)


class Salary(Income):
    def __init__(self, amount, date, description):
        super().__init__(amount, date, description)


class User:
    def __init__(self, user_id, username):
        self.user_id = user_id
        self.username = username
        self.transactions = []  # Store user transactions
        self.history_transactions = []  # Store history of transactions

    def add_transaction(self, transaction):
        self.transactions.append(transaction)
        self.history_transactions.append(transaction.get_attributes())

    def get_history_transactions(self):
        return self.history_transactions

    def get_total_income(self):
        return sum(transaction.amount for transaction in self.transactions if isinstance(transaction, Income))

    def get_total_expenses(self):
        return sum(transaction.amount for transaction in self.transactions if isinstance(transaction, Expense))

    def get_savings(self):
        return self.get_total_income() - self.get_total_expenses()

    def get_user_id(self):
        return self.user_id


class PersonalFinanceManager:
    def __init__(self):
        self.users = []
        self.users_id = []

    @property
    def get_users_id(self):
        return self.users_id
    @property
    def get_userslist(self):
        return self.users

    def create_user(self, user_id, username):
        new_user = User(user_id, username)
        self.users.append(new_user)
        self.users_id.append(new_user.get_user_id())
        return new_user

    def get_user(self, username):
        for user in self.users:
            if user.username == username:
                return user
        return None

    @property
    def get_users_transaction(self):
        # Return transactions for all users
        return [user.get_history_transactions() for user in self.users]

    def add_transaction_to_user(self, username, transaction):
        user = self.get_user(username)
        if user:
            user.add_transaction(transaction)
        else:
            print(f"User '{username}' not found.")

    def create_user(self, user_id, username):

        if user_id in self.users_id:
            print(f"User with user_id '{user_id}' already exists. Cannot create a new user.")
            return None


        existing_user = self.get_user(username)
        if existing_user:

            if existing_user.get_user_id() == user_id:
                print(
                    f"User with username '{username}' and user_id '{user_id}' already exists. Cannot create a new user.")
                return None

            # If the user has a different user_id, print a warning
            print(
                f"Warning: User with username '{username}' already exists with a different user_id. Creating a new user.")

        # Create a new user
        new_user = User(user_id, username)
        self.users.append(new_user)
        self.users_id.append(new_user.get_user_id())
        return new_user


pf_manager = PersonalFinanceManager()

user_ids = []
# open csv
with open('data_users.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)

    # read headers
    headers = next(csvreader)


    user_id_index = headers.index('user_id')
    user_income_index = headers.index('income')
    user_expense_index = headers.index('expense')
    user_description_index = headers.index('description')
    user_date_index = headers.index('date')
    user_name_index = headers.index('username')


    for row in csvreader:
        user_name = row[user_name_index]
        user_id = row[user_id_index]
        date = row[user_date_index]
        description = row[user_description_index]

        income = float(row[user_income_index])
        expense = float(row[user_expense_index])


        income_transaction = Income(income, date, description)
        expense_transaction = Expense(expense, date, description)


        if user_id not in user_ids:
            user = pf_manager.create_user(user_id, user_name)
            user_ids.append(user_id)
        else:
            users_id_list = pf_manager.get_users_id
            index = users_id_list.index(user_id)
            users_manager = pf_manager.get_userslist
            user = users_manager[index]

        user.add_transaction(income_transaction)
        user.add_transaction(expense_transaction)

data = []
for user in pf_manager.users:
    user_id = user.get_user_id()
    username = user.username

    savings_by_date = {}

    running_savings = 0


    for transaction in user.transactions:
        temp = True
        amount, date, description = transaction.get_attributes()
        income = amount if isinstance(transaction, Income) else 0
        expense = amount if isinstance(transaction, Expense) else 0
        saving = income - expense


        running_savings += saving

        savings_by_date[date] = running_savings
        for item in data:
            if date in item and user_id == item[0]:
                temp = False
                index = item.index(date)
                item[index + 1] += income
                item[index + 2] += expense
                item[index + 3] += saving

        if temp:
            data.append([user_id, username, date, income, expense, saving])
    for i in range(len(data) - 1):
        item = data[i]
        next_item = data[i + 1]
        if user_id == item[0] and user_id == next_item[0]:
            next_item[5] += item[5]
print(data)


columns = ["user_id", "username", "date", "income", "expense", "savings"]
transaction_df = pd.DataFrame(data, columns=columns)


transaction_df.to_csv('vizualization_data.csv', index=False)


print(transaction_df)

user_ids = transaction_df['user_id'].unique()
fig, axs = plt.subplots(len(user_ids), 3, figsize=(14, 5 * len(user_ids)))

if len(user_ids) == 1:
    user_id = user_ids[0]
    user_data = transaction_df[transaction_df['user_id'] == user_id]
    username = user_data['username'].iloc[0]
    axs[0].bar(user_data['date'], user_data['income'], color='green')
    axs[0].set_title(f'Total Income (Username: {username}, User ID: {user_id})')
    axs[0].set_xlabel('Date')
    axs[0].set_ylabel('Income')

    axs[1].bar(user_data['date'], user_data['expense'], color='red')
    axs[1].set_title(f'Total Expenses (Username: {username}, User ID: {user_id})')
    axs[1].set_xlabel('Date')
    axs[1].set_ylabel('Expenses')

    axs[2].plot(user_data['date'], user_data['savings'], marker='o', linestyle='-')
    axs[2].set_title(f'Savings (Username: {username}, User ID: {user_id})')
    axs[2].set_xlabel('Date')
    axs[2].set_ylabel('Savings')
else:
    for i, user_id in enumerate(user_ids):
        user_data = transaction_df[transaction_df['user_id'] == user_id]
        username = user_data['username'].iloc[0]

        axs[i, 0].bar(user_data['date'], user_data['income'], color='green')
        axs[i, 0].set_title(f'Total Income (Username: {username}, User ID: {user_id})')
        axs[i, 0].set_xlabel('Date')
        axs[i, 0].set_ylabel('Income')


        axs[i, 1].bar(user_data['date'], user_data['expense'], color='red')
        axs[i, 1].set_title(f'Total Expenses (Username: {username}, User ID: {user_id})')
        axs[i, 1].set_xlabel('Date')
        axs[i, 1].set_ylabel('Expenses')


        axs[i, 2].plot(user_data['date'], user_data['savings'], marker='o', linestyle='-')
        axs[i, 2].set_title(f'Savings (Username: {username}, User ID: {user_id})')
        axs[i, 2].set_xlabel('Date')
        axs[i, 2].set_ylabel('Savings')

plt.tight_layout()
plt.show()

[['1', 'Lilit', '1/1/2018', 82475.0, 11.11, 82463.89], ['36', 'Lilit', '1/2/2018', 32555.0, 1247.44, 31307.56], ['52', 'Lilit', '1/2/2018', 101738.0, 24.22, 101713.78], ['985', 'Lilit', '1/3/2018', 123424.0, 2298.09, 121125.91], ['973', 'Lilit', '1/4/2018', 171574.0, 11.76, 171562.24], ['961', 'Lilit', '1/5/2018', 100036.0, 25.85, 100010.15], ['949', 'Lilit', '1/6/2018', 143250.0, 18.45, 143231.55], ['937', 'Lilit', '1/8/2018', 150980.0, 45.0, 150935.0], ['925', 'Lilit', '1/8/2018', 192247.0, 15.38, 192231.62], ['913', 'Elena', '1/9/2018', 67932.0, 10.69, 67921.31], ['901', 'Elena', '1/10/2018', 280061.0, 89.46, 279971.54], ['889', 'Elena', '1/11/2018', 278036.0, 34.87, 278001.13], ['877', 'Elena', '1/11/2018', 19226.0, 43.54, 19182.46], ['865', 'Elena', '1/12/2018', 254359.0, 2000.0, 252359.0], ['853', 'Elena', '1/13/2018', 302834.0, 32.91, 302801.09], ['841', 'Elena', '1/13/2018', 647116.0, 39.05, 647076.95], ['829', 'Elena', '1/15/2018', 421878.0, 44.19, 421833.81], ['817', 'Elena',

ValueError: Image size of 1400x401000 pixels is too large. It must be less than 2^16 in each direction.

Error in callback <function _draw_all_if_interactive at 0x78b25ff9d000> (for post_execute):


ValueError: Image size of 1400x401000 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 1400x401000 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1400x401000 with 2406 Axes>